In [1]:
import nltk                                
import matplotlib.pyplot as plt            
import pandas as pd
import re                                  
import string                              
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer 

In [2]:
train = pd.read_csv('data.csv')
texts = list(train['Body'])
labels =list(train['Label'])

In [3]:
len(labels)

4009

In [4]:
len(texts)

4009

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.33, random_state=1)

In [6]:
import numpy as np
import pandas as pd

y_train = np.expand_dims(y_train, 1)
y_test = np.expand_dims(y_test, 1)

In [7]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)

(2686, 1)
(1323, 1)
(2686,)
(1323,)


In [9]:
import utils
import nltk
from os import getcwd
from utils import process_news, build_freqs

In [10]:
# create frequency dictionary
nltk.download('stopwords')
freqs = build_freqs(X_train, y_train)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


type(freqs) = <class 'dict'>
len(freqs) = 42802


In [11]:
print('This is an example of a True news: \n', X_train[0])
print('\nThis is an example of the processed version of the news: \n', process_news(X_train[0]))

This is an example of a True news: 
 President Trump and The First Lady Lead a Moment of Silence for Las Vegas
% of readers think this story is Fact. Add your two cents.
(Before It's News)
CDN – http://www.conservativedailynews.com
By Rich Mitchell
President Donald Trump and First Lady Melania Trump hold a moment of silence for the victims and families of those impacted by the Las Vegas shooting. The moment of silence will be observed at 2:45 PM EDT on the South Lawn of the White House.
The article President Trump and The First Lady Lead a Moment of Silence for Las Vegas is original content from Conservative Daily News.
CDN – http://www.conservativedailynews.com
Source: https://www.conservativedailynews.com/2017/10/president-trump-first-lady-lead-moment-silence/

This is an example of the processed version of the news: 
 ['presid', 'trump', 'first', 'ladi', 'lead', 'moment', 'silenc', 'la', 'vega', 'reader', 'think', 'stori', 'fact', 'add', 'two', 'cent', 'news', 'cdn', '–', 'rich', 'm

In [12]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))
    ### END CODE HERE ###
    
    return h

In [13]:
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


In [14]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]     
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of h
        h = sigmoid(z)
        
        # calculate the cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))                                                    

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [15]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.68359425.
The resulting vector of weights is [1.3e-07, 0.00010125, 6.196e-05]


In [26]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_news(tweet)
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [27]:
# Check your function

# test 1
# test on training data
tmp1 = extract_features(X_train[0], freqs)
print(tmp1)

[[1.0000e+00 2.2803e+04 2.6388e+04]]


In [28]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [29]:
X = np.zeros((len(X_train), 3))
for i in range(len(X_train)):
    X[i, :]= extract_features(X_train[i], freqs)

# training labels corresponding to X
Y = y_train

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 2000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

D:\Programs\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


The cost after training is nan.
The resulting vector of weights is [-1.4e-07, 0.00013072, -0.00016712]


In [30]:
def predict_news(news, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(news,freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [31]:
# Feel free to check the sentiment of your own tweet below
news = 'I am bad :)'
predict_news(news, freqs, theta)

array([[0.49771978]])

In [32]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    
    for news in test_x:
        # get the label prediction for the tweet
        y_pred = predict_news(news, freqs, theta)
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    ### END CODE HERE ###
    
    return accuracy

In [33]:
tmp_accuracy = test_logistic_regression(X_test, y_test, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.8700


In [34]:
# Some error analysis done for you
print('Label Predicted News')
for x,y in zip(X_test,y_test):
    y_hat = predict_news(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE News IS:', x)
        print('THE PROCESSED News IS:', process_news(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_news(x)).encode('ascii', 'ignore')))

Label Predicted News
THE News IS: Paul Anka Brings Vegas to Englewood, N.J.
(Before It's News)
CDN – http://www.conservativedailynews.com
By Jim Clayton
Paul Anka brought his talent and Vegas-like band to Bergen Performing Arts Center in Englewood, N.J. Wednesday night and what a show it was. Paul Anka is celebrating 60 years as an entertainer and songwriter this year, which makes him the only artist in history to have a song in the Billboard Top 100 during seven consecutive decades. Paul has shown himself to be the consummate, professional showman and entertainer. On Wednesday night he moved the audience from teary nostalgia to utter joy and hand clapping.
Backed by a 10 piece band consisting of a horn section with trumpet, Trombone and saxophone, percussion and bongo set, drums, three guitars, violin, keyboards and grand piano, it was truly a Vegas-style band and one of the best I’ve heard.
Paul opened up with Tom Jones’ song “She’s A Lady” followed by a jazzy version of “Mack the Kn

THE News IS: [gard align='center']
By Aaron Kesel
You can’t make this up. The IRS awarded Equifax with a multi-million dollar fraud prevention no-bid contract despite that its own execs committed market fraud when they sold shares before telling the public they were hacked.
The IRS will pay Equifax $7.25 million to help verify taxpayer identities and prevent fraud under a no-bid contract issued last week.
The credit agency will “verify taxpayer identity” and “assist in ongoing identity verification and validations” at the IRS, according to the award.
The award describes the contract as a “sole source order,” which means Equifax is the only company held capable of providing the service. The order was issued to prevent a lapse in identity checks while officials resolve a dispute over a separate contract.
Both Republican and Democrat lawmakers bashed the IRS decision as irresponsible and horrible.
“In the wake of one of the most massive data breaches in a decade, it’s irresponsible for th

THE News IS: Roger Federer has won 93 ATP titles - only Ivan Lendl (94) and Jimmy Connors (109) have won more
Rafael Nadal and Roger Federer both registered straightforward victories to move into the third round of the Shanghai Masters.
World number one Nadal needed just 54 minutes to beat America's Jared Donaldson 6-2 6-1.
Federer, chasing his 94th ATP title, found his rhythm as the match progressed to beat Argentine Diego Schwartzman 7-6 (7-4) 6-4.
Nadal plays Fabio Fognini next, while Federer faces Alexandr Dolgopolov.
Nadal, who leads world number two Federer in the rankings by 2,370 points, did not face a break point as he raced to victory over Donaldson.
Federer took a little longer to find his rhythm but 38 winners ensured his place in the second round.
Earlier, Britain's Aljaz Bedene was forced to retire from his second-round match against Germany's Alexander Zverev with a knee injury.
Bedene, ranked 48th in the world, was trailing 4-0 when he ended the match after just 12 minu

THE News IS: CRAIOVA, Romania (Reuters) - NATO launched a new multinational force in Romania on Monday to counter Russia along its eastern flank and to check a growing Russian presence in the Black Sea following the Kremlin’s 2014 seizure of Crimea.
The force will initially be built around a Romanian brigade of up to 4,000 soldiers, supported by troops from nine other NATO countries, and complementing a separate deployment of 900 U.S. troops who are already in place. The plans are to include additional air and sea assets to give the force greater capabilities.
“Our purpose is peace, not war,” Romanian President Klaus Iohannis told the NATO Parliamentary Assembly of alliance lawmakers, which is meeting this year in Bucharest.
“We are not a threat for Russia. But we need dialogue from a strong position of defense and discouragement,” he said, before flying to the Craiova military base in south-eastern Romania.
At the base, as military bands played, NATO Secretary-General Jens Stoltenberg

THE News IS: FILE PHOTO: Shania Twain poses on the red carpet a the Billboard Magazine's 11th annual Women in Music luncheon in New York, NY, U.S. on December 9, 2016. REUTERS/Shannon Stapleton/File Photo
LOS ANGELES (Reuters) - Shania Twain’s first studio album in 15 years debuted at the top of the Billboard album charts, while rocker Tom Petty’s death sent his music soaring back onto the U.S. charts.
Twain’s album “Now” sold more than 137,000 copies in its first week, according to data on Monday from Nielsen SoundScan, almost double new releases by pop singers Demi Lovato and Miley Cyrus.
Petty’s “Greatest Hits” album with his band The Heartbreakers, first released in 1993, sold some 83,000 units to take the No. 2 spot on the Billboard 200 a week after the “American Girl” singer died at age 66.
The Billboard 200 chart tallies units from album sales, song sales (10 songs equal one album) and streaming activity (1,500 streams equal one album).
Canadian country-pop crosssover Twain, 52,

...

THE News IS: Photo
A biopic can take any life, no matter how transgressive or interesting, and reduce it to check marks on a list. In “Tom of Finland,” a primarily Finnish-language feature from Dome Karukoski, the biography is that of Touko Laaksonen (Pekka Strang), the Finnish advertising designer who, under the name Tom of Finland, published drawings of hyper-muscular, well-endowed men and became famous for expanding perceptions on the portrayal of gay life in art.
While Mr. Laaksonen devoted his life (1920-91) to challenging conventions, the film is committed to honoring them. The movie traces Mr. Laaksonen’s service in World War II, his experience with public-park cruising and discrimination, and his early illustrations of aggrandized masculinity. (He says the pictures would be easier to publish in the Vatican than in Finland, where homosexuality was outlawed at the time.) His sister (Jessica Grabowsky), a fellow artist, suspicious of his sexuality, dares him to kiss a woman in a g

THE News IS: The Pyramid of Cheops, the most beautiful revelation of all times...
 
Pyramid Apocalypsia. Published on September 23, 2017, by Jean Seimple.
 
At this moment or your eyes are focused on his words, you do not yet know that your vision of the world will probably change forever.
 
Now your mind is in the future because you are wondering what I will be able to reveal to you. And there it is in the past because you are perhaps thinking back to what you just learned...
 
Yes, take some of your time for the Architect of the Great Pyramid, for, believe me, no one deserves it as much as he does...
 
More than two years ago, we revealed to you that the plan of the pyramid was a very precise map of the Earth, that the chambers symbolized their house before and after a catastrophe and indicated their respective geographical positions.
 
<img src="https://image.jimcdn.com/app/cms/image/transf/dimension=450x10000:format=jpg/path/s55b662ef4a1e632a/image/ia5d1022e8540004f/version/1506037

0	0.68858641	b"pyramid cheop beauti revel time ... pyramid apocalypsia publish septemb 23 2017 jean seimpl moment eye focus word yet know vision world probabl chang forev mind futur wonder abl reveal past perhap think back learn ... ye take time architect great pyramid believ one deserv much ... two year ago reveal plan pyramid precis map earth chamber symbol hous catastroph indic respect geograph posit img src reconstitut neolith hous j roethlisberg  latnium neuchtel 1 relat articl unlock truth 2014 georgian guideston bombshel nootrop key unlock human brain potenti hous found antarctica formerli call atlanti thank coordin provid pyramid nasa radar map locat ice antarct two citi correspond exactli descript given plato img src also reveal meridian greenwich secretli commemor present posit atlanti meridian pyramid indic previou posit transmit warn messag pyramid secret link pyramid kheop bibl well dozen inform ... today make anoth revel ... beauti revel time ... learn articl simpli under

THE PROCESSED News IS: ['oct', '9', '2017', 'chicago', 'il', 'usa', 'minnesota', 'vike', 'kicker', 'kai', 'forbath', '2', 'kick', 'game', 'win', 'field', 'goal', 'chicago', 'bear', 'vike', 'punter', 'ryan', 'quigley', '4', 'hold', 'second', 'half', 'soldier', 'field', 'mandatori', 'credit', 'matt', 'marton-usa', 'today', 'sport', 'sport', 'xchang', 'kai', 'forbath', 'kick', '26', 'yard', 'field', 'goal', '12', 'second', 'remain', 'lift', 'minnesota', 'vike', '20-17', 'win', 'chicago', 'bear', 'monday', 'vike', 'quarterback', 'case', 'keenum', 'came', 'bench', 'complet', '17', '21', 'pass', '140', 'yard', 'touchdown', 'replac', 'sam', 'bradford', 'struggl', 'badli', 'first', 'half', 'minnesota', '3-2', 'earn', 'first', 'road', 'win', 'season', 'bear', 'quarterback', 'mitchel', 'trubiski', 'complet', '12', '25', 'pass', '119', 'yard', 'touchdown', 'nfl', 'debut', 'rooki', 'threw', 'intercept', 'vike', 'safeti', 'harrison', 'smith', 'deep', 'bear', 'territori', 'set', 'go-ahead', 'score',

THE News IS: (Before It's News)
This is terrorism. No question.
 
 
Imagine if those fuel tanks blew. Paddock would have blown up the city. The Las Vegas sheriff said that Paddock could not have done this alone.
 
Las Vegas Strip shooter targeted aviation fuel tanks, source says
Emigrate While You Still Can! Learn More...
 
 Review Journal, October 4, 2017:
 
 
 
Las Vegas Strip mass murderer Stephen Paddock used his Mandalay Bay hotel room to spray massive aviation fuel tanks with bullets Sunday night, a knowledgeable source told the Las Vegas Review-Journal.
 
The bullets left holes, but did not penetrate the two circular white tanks, sparing the nearby Route 91 Harvest country music festival from a potentially massive explosion, the source said Wednesday.
 
The tanks are roughly 1,100 feet from the concert site, where Paddock killed 58 people and wounded almost 500.
 
Within the past couple of days, construction crews repaired and painted the holes, and FBI agents inspected the tank

In [36]:
news = X_test[0]
#print(process_tweet(my_tweet))
y_hat = predict_news(news, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('True')
else: 
    print('Fake')

[[0.59772297]]
True


In [37]:
y_test[0]

array([1])

In [38]:
X_test[0]

"FILE PHOTO: An employee is seen behind a glass wall with the logo of Alibaba at the company's headquarters on the outskirts of Hangzhou, Zhejiang province, April 23, 2014. REUTERS/Chance Chan/File Photo\nLONDON (Reuters) - The boom in emerging market technology stocks is becoming a problem for fund managers of all stripes.\nThe soaring market capitalization of a handful of companies such as China’s Alibaba (BABA.N) and Tencent (0700.HK) is steadily lifting their weighting in the MSCI emerging equities index .MSCIEF.\nThis means investors in funds that track indexes (exchange traded funds or ETFs) - who want exposure to a range of companies for a lower fund management fee - are finding themselves increasingly exposed to a single sector.\nMeanwhile, active fund managers, who justify charging higher fees for their individual stock-picking expertise, are under pressure to buy those tech stocks to ensure their funds keep up with the index’s gains.\nAnd with both sets of investor chasing th